In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import torch
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')





In [ ]:
os.listdir(os.path.join(RESULTPATH, 'match_ratios'))

In [ ]:
match_ratios = h5py.File(os.path.join(RESULTPATH, 'match_ratios', '220114_sens_est_dense_grid_87.0_0cm_match_ratios.h5'), 'r')

In [ ]:
match_ratios['1024'].shape

# plot histograms

In [ ]:
# choose a down selection factor
n_down_select = 0

# choose a number of samples
n_sample = 8192

match = match_ratios[str(n_sample)][n_down_select, :]

plt.figure()

hist = plt.hist(match, 64, )

# plot series of histograms for different down selections

In [ ]:
# choose a number of samples
n_sample = 8192

# choose a series of down-selection factors
n_down_select_list = [0]

# choose a number of bins
n_bins = [64,]

# choose a histogram style
hist_type = 'step'

# plotting params
figsize = (13, 8)
xlimits = [0.4, 1.01]


sns.set_theme(context='poster')
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(1,1,1,)


for i, n_down_select in enumerate(n_down_select_list):
    match = match_ratios[str(n_sample)][n_down_select, :]
    grid_size = match_ratios[str(n_sample)].attrs['number-of-templates'][n_down_select]
    mean_match = int(match.mean() * 1000) / 1000
    #print(np.std(match))
    hist = ax.hist(match, n_bins[i] , histtype=hist_type, label = f'Number of Templates = {grid_size}, Mean = {mean_match}', range = (0.0, 1.01))
    
ax.set_xlabel('Match (Best Score / Ideal Score)')
ax.set_ylabel('N')

plt.legend(loc=2)

name = '220114_match_histograms_84.5_0cm_modify_grid_size.png'
save_path = os.path.join(PATH, 'plots', 'mayfly', name)

plt.tight_layout()
#plt.savefig(save_path)

# plot a series of match histograms for different track lengths

In [ ]:
# choose a series numbers of samples
n_sample_list = [1024, 2048, 4096, 8192]

# choose a down-selection factor
n_down_select = 132

# choose a number of bins
n_bins = [64, 64, 64, 64]

# choose a histogram style
hist_type = 'step'

# plotting params
figsize = (13, 8)
xlimits = [0.4, 1.01]


sns.set_theme(context='poster')
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(1,1,1,)


for i, n_sample in enumerate(n_sample_list):
    match = match_ratios[str(n_sample)][n_down_select, :]
    grid_size = match_ratios[str(n_sample)].attrs['number-of-templates'][n_down_select]
    mean_match = int(match.mean() * 1000) / 1000
    #print(np.std(match))
    hist = ax.hist(match, n_bins[i] , histtype=hist_type, label = f'Number of Samples = {2 * n_sample}, Mean = {mean_match}', range = (.8, 1.01))
    
ax.set_xlabel('Match (Best Score / Ideal Score)')
ax.set_ylabel('N')

plt.legend(loc=2)

name = '220114_match_histograms_87.0_0cm_modify_sample_number.png'
save_path = os.path.join(PATH, 'plots', 'mayfly', name)

plt.tight_layout()
#plt.savefig(save_path)

# plot mean match vs template number for several different sample lengths

In [ ]:
# choose a number of samples
n_sample_list = [1024, 2048, 4096, 8192]

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for n_sample in n_sample_list:
    
    match = match_ratios[str(n_sample)][:].mean(axis=-1)
    num_templates = match_ratios[str(n_sample)].attrs['number-of-templates']


    smooth_kernel = np.ones(21) / 21

    #smooth_match = scipy.signal.convolve(match, smooth_kernel,  mode='same')

    ax.plot(num_templates, match, '.', label = f'Sample Length = {2 * n_sample}')

ax.set_xscale('log')
ax.set_xlabel('Number of Templates')
ax.set_ylabel('Mean Match (Best Score / Ideal Score)')
plt.legend(loc=4)


name = '220114_mean_match_vs_number_of_templates_84.5_0cm_modify_sample_number.png'
save_path = os.path.join(PATH, 'plots', 'mayfly', name)

plt.tight_layout()

plt.savefig(save_path)

In [ ]:
plt.plot(num_templates)
plt.yscale('log')

In [ ]:
print(num_templates)

# Down select template grids

In [ ]:


energies_base = np.linspace(18575, 18580, 101)
angles_base = np.linspace(89.5 - 0.005, 89.5 + 0.005, 101)

n_point = 2

energy_grid, angle_grid = np.meshgrid(energies_base, angles_base)

#for i in range(9):

#    print(energy_grid.flatten()[np.arange(0, 101 * 101, i+1)].size)

fig = plt.figure(figsize = (13, 13))

ax = fig.add_subplot(1,1,1)
ax.plot(energy_grid.flatten()[np.arange(0, 101 * 101, n_point)], angle_grid.flatten()[np.arange(0, 101 * 101, n_point)], '.', markersize=10)

ax.set_xlabel('Energy (eV)')
ax.set_ylabel('Pitch Angle (deg)')

ax.set_title(f'Example Down-selected Grid, N = {energy_grid.flatten()[np.arange(0, 101 * 101, n_point)].size}')

name = '220111_example_down_select_grid_4'

#plt.savefig(os.path.join(PATH, 'plots', 'mayfly', name))

# animate grids

In [ ]:
sns.set_theme(context='poster')

energies_base = np.linspace(18575, 18580, 101)
angles_base = np.linspace(89.5 - 0.005, 89.5 + 0.005, 101)

energy_grid, angle_grid = np.meshgrid(energies_base, angles_base)

def animate(k):
    line.set_xdata(energy_grid.flatten()[np.arange(0, 101 * 101, k+1)])
    line.set_ydata(angle_grid.flatten()[np.arange(0, 101 * 101, k+1)])
    
    ax.set_title(k)
    
    return line,


fig = plt.figure(figsize = (13, 13))

ax = fig.add_subplot(1,1,1)

line, = ax.plot(energy_grid.flatten(), angle_grid.flatten(), '.', markersize=10)
ax.set_title(1)

#ani = matplotlib.animation.FuncAnimation(fig, animate, frames = np.arange(1, 1000, 20), interval=700)

#ani.save(os.path.join(PATH, 'plots', 'mayfly', 'test.gif', ))

# Compute Mean Matches for different track lengths, save data

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores', ))

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores', '220112_sens_est_dense_grid_89.5_0cm_template_scores_random_nslice2_modify_nsample'))